In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit import Gate
from qiskit.circuit.library.standard_gates import UGate

class DisplacementGate(Gate):
    """
    Phase space displacement gate for continuous variables.

    D(a, phi) = exp(-i * sqrt(2/hbar) * (re(alpha) * p - im(alpha) * x))

    where alpha = a * exp(i * phi).
    """

    def __init__(self, a, phi):
        # Initialize as a gate with 2 parameters (a and phi) acting on 1 qubit
        super().__init__("D", 1, [a, phi])

    def _define(self):
        """
        Define the decomposition of this gate in terms of more fundamental gates.
        """
        a, phi = self.params

        # Creating the corresponding unitary matrix for the displacement gate
        D_matrix = np.array([
            [1, 0, 0],
            [2 * a * np.cos(phi), 1, 0],
            [2 * a * np.sin(phi), 0, 1]
        ])

        # Add this unitary as a custom gate to the quantum circuit
        q = QuantumCircuit(1)
        q.unitary(D_matrix, 0)

        self.definition = q

    def inverse(self):
        """Returns the adjoint (inverse) of the displacement gate."""
        a, phi = self.params
        new_phi = (phi + np.pi) % (2 * np.pi)
        return DisplacementGate(a, new_phi)

class DisplacementEmbedding:
    def __init__(self, num_qubits: int, amplitude: bool = True):
        """
        Class for embedding a data vector into a quantum circuit using DisplacementGates.

        Args:
            num_qubits (int): The number of qubits in the circuit.
            amplitude (bool): If True, the amplitude will be parameterized and the phase will be fixed at 0.1.
                              If False, the phase will be parameterized and the amplitude will be fixed at 0.1.
        """
        self.num_qubits = num_qubits
        self.amplitude = amplitude
        self.parameters = ParameterVector('x', length=num_qubits)

    def create_circuit(self):
        """
        Creates and returns a quantum circuit with the specified DisplacementGates applied to each qubit.

        Returns:
            QuantumCircuit: The generated quantum circuit.
        """
        qc = QuantumCircuit(self.num_qubits)

        for i in range(self.num_qubits):
            if self.amplitude:
                # Parameterize the amplitude with fixed phase
                amplitude = self.parameters[i]
                phase = 0.1
            else:
                # Parameterize the phase with fixed amplitude
                amplitude = 0.1
                phase = self.parameters[i]

            displacement_gate = DisplacementGate(amplitude, phase)
            qc.append(displacement_gate, [i])

        return qc

In [2]:
embedding = DisplacementEmbedding(num_qubits=3, amplitude=True)
circuit = embedding.create_circuit()
circuit.draw()

┌─────────────┐
q_0: ┤ D(x[0],0.1) ├
     ├─────────────┤
q_1: ┤ D(x[1],0.1) ├
     ├─────────────┤
q_2: ┤ D(x[2],0.1) ├
     └─────────────┘